#INSTALL

In [3]:
pip install numpy-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.9/239.9 kB 2.2 MB/s eta 0:00:00


#IMPORTS

In [2]:
from torch_geometric.data import Data
from torch_geometric.utils import convert
import torch
import pandas as pd
import torch_geometric.transforms as T
#import tensorflow as tf
import itertools
import numpy as np
import torch
from torch.nn import Linear
import seaborn as sns
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import HeteroData

from torch_geometric.utils import negative_sampling

from torch_geometric.nn import ChebConv
from torch_geometric.nn import HypergraphConv
import torch.nn.functional as F

#FOR VISUALIZING GRAPH
import matplotlib.pyplot as plt
import networkx as nx
import random
from collections import Counter
from torch_geometric.utils import to_networkx


from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay, roc_curve
from sklearn.metrics import DetCurveDisplay
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve

In [3]:
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows=999
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#FUNCTION CODES

In [4]:
class Net(torch.nn.Module):

    def __init__(self, in_channels, hidden1_channels, hidden2_channels,out_channels,dec,af_val,num_layers,epoch,aggr,var):

        super().__init__()
        self.convs = torch.nn.ModuleList()
        if(var=="ChebConv"):
          self.convs.append(eval(var)(in_channels, hidden1_channels,aggr=aggr,K=3))
          for _ in range(num_layers - 2):
              self.convs.append(eval(var)(hidden1_channels, hidden1_channels,aggr=aggr,K=3))
          self.convs.append(eval(var)(hidden1_channels, out_channels,aggr=aggr,K=3))
        else:
          self.convs.append(eval(var)(in_channels, hidden1_channels,aggr=aggr))
          for _ in range(num_layers - 2):
              self.convs.append(eval(var)(hidden1_channels, hidden1_channels,aggr=aggr))
          self.convs.append(eval(var)(hidden1_channels, out_channels,aggr=aggr))


    def encode(self, x, edge_index,af_val):
        prev_x = None
        x = x.to(device)
        edge_index = edge_index.to(device)   
        for i,conv in enumerate(self.convs[:-1]):
            prev_x = x
            x = conv(x, edge_index)
            if i > 0:
              x = x + prev_x
            x = eval(af_val)(x)
            # x = F.dropout(x, p=.2, training=self.training)
        x = self.convs[-1](x, edge_index)
        return x


    def decode(self, z, edge_label_index,dec):
        '''TYPE 1 - Multiplying and adding node embeddings'''
        '''TYPE 2 - Cosine similarity'''
        '''TYPE 2 - Neural Network'''
        if(dec=="dot_sum") :
          return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1)
        else:
          cos = torch.nn.CosineSimilarity(dim=1)
          output = cos(z[edge_label_index[0]].float(), z[edge_label_index[1]].float())
          return output


def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs,af_val,dec
):

    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index,af_val)
        positive_num = train_data.edge_index.shape[1]
        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index,
            num_nodes=train_data.num_nodes,
            method='sparse'
        ).to(device)
        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        ).to(device)
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0).to(device)


        out1 = model.decode(z, edge_label_index,dec)
        out = out1.view(-1)
        loss = criterion(out, edge_label.float())
        loss.backward()
        optimizer.step()

        val_auc,precision, recall,fpr, tpr, mcc, jac_score, cohkap_score, f1, top_k = eval_link_predictor(model, val_data,af_val,dec)

    return model


@torch.no_grad()
def eval_link_predictor(model, data,af_val,dec):
    model.eval()
    z = model.encode(data.x.to(device), data.edge_index.to(device), af_val)
    out = model.decode(z, data.edge_label_index.to(device), dec)#.view(-1)
    actual = data.edge_label.cpu().numpy()
    auc = roc_auc_score(actual, out.cpu().numpy())
    fpr, tpr, _ = roc_curve(actual, out.cpu().numpy())
    precision, recall, thresholds = precision_recall_curve(actual, out.cpu().numpy())

    pred =out.cpu().numpy()
    pred[pred > 0.5] = 1
    pred[pred <= 0.5] = 0

    mcc = matthews_corrcoef(actual, pred)
    jac_score = jaccard_score(actual, pred)
    cohkap_score = cohen_kappa_score(actual, pred)
    f1 = f1_score(actual, pred)
    top_k = top_k_accuracy_score(actual, pred, k=1)

    return auc,precision, recall,fpr, tpr, mcc, jac_score, cohkap_score, f1, top_k

@torch.no_grad()
def prediction(model, data,af_val,dec):
    model.eval()
    z = model.encode(data.x.to(device), data.edge_index.to(device), af_val)
    out = model.decode(z, data.edge_label_index.to(device), dec)#.view(-1)
    pred =out.cpu().numpy()
    pred[pred > 0.5] = 1
    pred[pred <= 0.5] = 0
    print(pred)
    return pred


In [5]:
def main_run(folder_name,file_hetero,file_gold,hidden1_channels,hidden2_channels,out_channels,data,org,ds,dec,af_val,num_layers,epoch,aggr,var):
  auprs =0
  aucs =0

  for i in range(10) :
    # print("i is : ", i)
    split = T.RandomLinkSplit(
        num_val=0.2,
        num_test=0.3,
        is_undirected=False,
        add_negative_train_samples=True,
        key="edge_label"
        # edge_types=('source', 'interaction', 'target'),
        # neg_sampling_ratio=8684
    )

    train_data, val_data, test_data = split(data)
    index=(train_data.edge_label == 1).nonzero(as_tuple=True)[0]
    ''' OLD CODE'''
    in_channels=data.num_features

    model =  Net(in_channels, hidden1_channels, hidden2_channels, out_channels,dec,af_val,num_layers,epoch,aggr,var).to(device)#Net(data.num_features, data.num_features, 128, 64).to(device) #self, in_channels, hidden1_channels, hidden2_channels,out_channels
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)#RMSprop(params=model.parameters())#
    criterion = torch.nn.BCEWithLogitsLoss()
    model = train_link_predictor(model, train_data, val_data, optimizer, criterion,epoch,af_val,dec).to(device)

    test_auc, precision, recall,fpr, tpr, mcc, jac_score, cohkap_score, f1, top_k = eval_link_predictor(model, test_data,af_val,dec)


    # tprs.append(tpr)
    # fprs.append(fpr)
    aucs = aucs+test_auc
    aupr = auc(recall, precision)
    auprs=auprs+aupr

  mean_auc = float(aucs/10)
  mean_aupr = float(auprs/10)


  list1 = [org, ds, dec, af_val,num_layers, epoch, aggr, var,mean_auc, mean_aupr, mcc, jac_score,cohkap_score, f1, top_k]
  df = pd.DataFrame(list1).T
  df.columns = ["org", "ds", "dec", "af_val","num_layers", "epoch", "aggr", "var","auc", "aupr", "mcc", "jac_score","cohkap_score", "f1", "top_k"]
  #
  return df,model


#DREAM5 - MAIN

In [12]:

path = "/content/drive/MyDrive/Colab Notebooks/data/DREAM5/training data/"
gold_std = "/content/drive/MyDrive/Colab Notebooks/data/DREAM5/gold/test data/"

organism  = {"Insilico":  ["InSilicoSize100","net1_expression_data.tsv","DREAM5_NetworkInference_GoldStandard_Network1 - in silico.tsv"],
                        # "Saure":  ["InSilicoSize100","net2_expression_data.tsv","DREAM5_NetworkInference_GoldStandard_Network2 - S. aureus.txt"],
                        "Ecoli":  ["InSilicoSize100","net3_expression_data.tsv","DREAM5_NetworkInference_GoldStandard_Network3 - E. coli.tsv"],
                        "Scere":  ["InSilicoSize100","net4_expression_data.tsv","DREAM5_NetworkInference_GoldStandard_Network4 - S. cerevisiae.tsv"]
                            }


parameters = {
    "decoder": ["dot_sum"],# substitute "cos" to use cos decoder
    "af": ["F.sigmoid"],  # substitute "F.silu","F.tanh" for activation functions
    "num_layers" : [3],#2,4,5 #substitute layer count
    "variant" : ["HypergraphConv"],#,"SSGConv","ChebConv","ClusterGCNConv" #substitute convolution layers
    "aggrs" :["sum"],# substitute "add"
    "epochs" :[200]# substitute epochs 100,150,200,250
               }



hidden1_channels=128
hidden2_channels=64
out_channels= 32
num_layers = 4
final_out = pd.DataFrame()

ds_type =["basic_aug_data_"] # Try different graph types : "basic_TS_data_","basic_TS_aug_data_","basic_data_"
for org, files in organism.items():
  # final_out = pd.DataFrame()
  for ds in ds_type:
    for dec in parameters.get('decoder'):
      for lay_num in parameters.get('num_layers'):
        for epoch in parameters.get('epochs'):
          for af_val in parameters.get('af'):
            for aggr in parameters.get('aggrs'):
              for var in parameters.get('variant'):
                # for dim in parameters.get('hc'):
                  print("current location : "+str(org)+'_'+str(ds)+'_'+str(dec)+'_'+str(lay_num)+'_'+str(epoch)+'_'+str(af_val)+'_'+str(aggr)+'_'+str(var))
                  print(path+ds+org+"_"+files[0]+".pt")
                  data = torch.load(path+ds+org+"_"+files[0]+".pt")#.cuda()

                  #GSE1297 = torch.load("/content/drive/MyDrive/Colab Notebooks/DREAM challenge GCN GNN Experimentation/Human.pt").cuda()
                  temp,model = main_run(files[0],files[1],files[2],hidden1_channels,hidden2_channels,out_channels,data,org,ds,dec,af_val,lay_num,epoch,aggr,var)
                  #pred = prediction(model, GSE1297,af_val,dec)
                  final_out = final_out.append(temp)

print(final_out)
# final_out.to_excel('/content/drive/MyDrive/Colab Notebooks/DREAM challenge GCN GNN Experimentation/DREAM5_InsilicoSize100_'+str(org)+'_'+var+'Dimension_count.xlsx', index=False)

current location : Insilico_basic_aug_data__dot_sum_3_200_F.sigmoid_sum_HypergraphConv
/content/drive/MyDrive/Colab Notebooks/data/DREAM5/training data/basic_aug_data_Insilico_InSilicoSize100.pt
current location : Ecoli_basic_aug_data__dot_sum_3_200_F.sigmoid_sum_HypergraphConv
/content/drive/MyDrive/Colab Notebooks/data/DREAM5/training data/basic_aug_data_Ecoli_InSilicoSize100.pt
current location : Scere_basic_aug_data__dot_sum_3_200_F.sigmoid_sum_HypergraphConv
/content/drive/MyDrive/Colab Notebooks/data/DREAM5/training data/basic_aug_data_Scere_InSilicoSize100.pt
        org               ds      dec     af_val num_layers epoch aggr  \
0  Insilico  basic_aug_data_  dot_sum  F.sigmoid          3   200  sum   
0     Ecoli  basic_aug_data_  dot_sum  F.sigmoid          3   200  sum   
0     Scere  basic_aug_data_  dot_sum  F.sigmoid          3   200  sum   

              var       auc      aupr       mcc jac_score cohkap_score  \
0  HypergraphConv  0.878696  0.865001  0.659332  0.66950

##Version details

absl-py==1.4.0
aiohttp==3.9.3
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.22.1
bigframes==0.20.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.42
branca==0.7.1
build==1.0.3
CacheControl==0.14.0
cachetools==5.3.2
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.7
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.0
colorcet==3.0.1
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.0
cryptography==42.0.2
cufflinks==0.17.3
cupy-cuda12x==12.2.0
cvxopt==1.3.2
cvxpy==1.3.3
cycler==0.12.1
cymem==2.0.8
Cython==3.0.8
dask==2023.8.1
datascience==0.17.6
db-dtypes==1.2.0
dbus-python==1.2.18
debugpy==1.6.6
decorator==4.4.2
defusedxml==0.7.1
diskcache==5.6.3
distributed==2023.8.1
distro==1.7.0
dlib==19.24.2
dm-tree==0.1.8
docutils==0.18.1
dopamine-rl==4.0.6
duckdb==0.9.2
earthengine-api==0.1.388
easydict==1.11
ecos==2.0.12
editdistance==0.6.2
eerepr==0.0.4
en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl#sha256=86cc141f63942d4b2c5fcee06630fd6f904788d2f0ab005cce45aadb8fb73889
entrypoints==0.4
et-xmlfile==1.1.0
etils==1.6.0
etuples==0.3.9
exceptiongroup==1.2.0
fastai==2.7.14
fastcore==1.5.29
fastdownload==0.0.7
fastjsonschema==2.19.1
fastprogress==1.0.3
fastrlock==0.8.2
filelock==3.13.1
fiona==1.9.5
firebase-admin==5.3.0
Flask==2.2.5
flatbuffers==23.5.26
flax==0.8.0
folium==0.14.0
fonttools==4.47.2
frozendict==2.4.0
frozenlist==1.4.1
fsspec==2023.6.0
future==0.18.3
gast==0.5.4
gcsfs==2023.6.0
GDAL==3.6.4
gdown==4.7.3
geemap==0.30.4
gensim==4.3.2
geocoder==1.38.1
geographiclib==2.0
geopandas==0.13.2
geopy==2.3.0
gin-config==0.5.0
glob2==0.7
google==2.0.3
google-ai-generativelanguage==0.4.0
google-api-core==2.11.1
google-api-python-client==2.84.0
google-auth==2.17.3
google-auth-httplib2==0.1.1
google-auth-oauthlib==1.2.0
google-cloud-aiplatform==1.39.0
google-cloud-bigquery==3.12.0
google-cloud-bigquery-connection==1.12.1
google-cloud-bigquery-storage==2.24.0
google-cloud-core==2.3.3
google-cloud-datastore==2.15.2
google-cloud-firestore==2.11.1
google-cloud-functions==1.13.3
google-cloud-iam==2.14.0
google-cloud-language==2.9.1
google-cloud-resource-manager==1.12.0
google-cloud-storage==2.8.0
google-cloud-translate==3.11.3
google-colab @ file:///colabtools/dist/google-colab-1.0.0.tar.gz#sha256=1d59f6b64dadf8ce0c1c2be96eab7ab76eb09ec2ef3192d0feb07e15221763b3
google-crc32c==1.5.0
google-generativeai==0.3.2
google-pasta==0.2.0
google-resumable-media==2.7.0
googleapis-common-protos==1.62.0
googledrivedownloader==0.4
graphviz==0.20.1
greenlet==3.0.3
grpc-google-iam-v1==0.13.0
grpcio==1.60.1
grpcio-status==1.48.2
gspread==3.4.2
gspread-dataframe==3.3.1
gym==0.25.2
gym-notices==0.0.8
h5netcdf==1.3.0
h5py==3.9.0
holidays==0.41
holoviews==1.17.1
html5lib==1.1
httpimport==1.3.1
httplib2==0.22.0
huggingface-hub==0.20.3
humanize==4.7.0
hyperopt==0.2.7
ibis-framework==7.1.0
idna==3.6
imageio==2.31.6
imageio-ffmpeg==0.4.9
imagesize==1.4.1
imbalanced-learn==0.10.1
imgaug==0.4.0
importlib-metadata==7.0.1
importlib-resources==6.1.1
imutils==0.5.4
inflect==7.0.0
iniconfig==2.0.0
install==1.3.5
intel-openmp==2023.2.3
ipyevents==2.0.2
ipyfilechooser==0.6.0
ipykernel==5.5.6
ipyleaflet==0.18.2
ipython==7.34.0
ipython-genutils==0.2.0
ipython-sql==0.5.0
ipytree==0.2.2
ipywidgets==7.7.1
itsdangerous==2.1.2
jax==0.4.23
jaxlib @ https://storage.googleapis.com/jax-releases/cuda12/jaxlib-0.4.23+cuda12.cudnn89-cp310-cp310-manylinux2014_x86_64.whl#sha256=8e42000672599e7ec0ea7f551acfcc95dcdd0e22b05a1d1f12f97b56a9fce4a8
jeepney==0.7.1
jieba==0.42.1
Jinja2==3.1.3
joblib==1.3.2
jsonpickle==3.0.2
jsonschema==4.19.2
jsonschema-specifications==2023.12.1
jupyter-client==6.1.12
jupyter-console==6.1.0
jupyter-server==1.24.0
jupyter_core==5.7.1
jupyterlab-widgets==3.0.9
jupyterlab_pygments==0.3.0
kaggle==1.5.16
kagglehub==0.1.8
keras==2.15.0
keyring==23.5.0
kiwisolver==1.4.5
langcodes==3.3.0
launchpadlib==1.10.16
lazr.restfulclient==0.14.4
lazr.uri==1.0.6
lazy_loader==0.3
libclang==16.0.6
librosa==0.10.1
lida==0.0.10
lightgbm==4.1.0
linkify-it-py==2.0.2
llmx==0.0.15a0
llvmlite==0.41.1
locket==1.0.0
logical-unification==0.4.6
lxml==4.9.4
malloy==2023.1067
Markdown==3.5.2
markdown-it-py==3.0.0
MarkupSafe==2.1.5
matplotlib==3.7.1
matplotlib-inline==0.1.6
matplotlib-venn==0.11.10
mdit-py-plugins==0.4.0
mdurl==0.1.2
miniKanren==1.0.3
missingno==0.5.2
mistune==0.8.4
mizani==0.9.3
mkl==2023.2.0
ml-dtypes==0.2.0
mlxtend==0.22.0
more-itertools==10.1.0
moviepy==1.0.3
mpmath==1.3.0
msgpack==1.0.7
multidict==6.0.5
multipledispatch==1.0.0
multitasking==0.0.11
murmurhash==1.0.10
music21==9.1.0
natsort==8.4.0
nbclassic==1.0.0
nbclient==0.9.0
nbconvert==6.5.4
nbformat==5.9.2
nest-asyncio==1.6.0
networkx==3.2.1
nibabel==4.0.2
nltk==3.8.1
notebook==6.5.5
notebook_shim==0.2.3
numba==0.58.1
numexpr==2.9.0
numpy==1.23.5
numpy-ml==0.1.2
oauth2client==4.1.3
oauthlib==3.2.2
opencv-contrib-python==4.8.0.76
opencv-python==4.8.0.76
opencv-python-headless==4.9.0.80
openpyxl==3.1.2
opt-einsum==3.3.0
optax==0.1.8
orbax-checkpoint==0.4.4
osqp==0.6.2.post8
packaging==23.2
pandas==1.5.3
pandas-datareader==0.10.0
pandas-gbq==0.19.2
pandas-stubs==1.5.3.230304
pandocfilters==1.5.1
panel==1.3.8
param==2.0.2
parso==0.8.3
parsy==2.1
partd==1.4.1
pathlib==1.0.1
patsy==0.5.6
peewee==3.17.0
pexpect==4.9.0
pickleshare==0.7.5
Pillow==9.4.0
pins==0.8.4
pip-tools==6.13.0
platformdirs==4.2.0
plotly==5.15.0
plotnine==0.12.4
pluggy==1.4.0
polars==0.20.2
pooch==1.8.0
portpicker==1.5.2
prefetch-generator==1.0.3
preshed==3.0.9
prettytable==3.9.0
proglog==0.1.10
progressbar2==4.2.0
prometheus-client==0.19.0
promise==2.3
prompt-toolkit==3.0.43
prophet==1.1.5
proto-plus==1.23.0
protobuf==3.20.3
psutil==5.9.5
psycopg2==2.9.9
ptyprocess==0.7.0
py-cpuinfo==9.0.0
py4j==0.10.9.7
pyarrow==10.0.1
pyarrow-hotfix==0.6
pyasn1==0.5.1
pyasn1-modules==0.3.0
pycocotools==2.0.7
pycparser==2.21
pyct==0.5.0
pydantic==2.6.0
pydantic_core==2.16.1
pydata-google-auth==1.8.2
pydot==1.4.2
pydot-ng==2.0.0
pydotplus==2.0.2
PyDrive==1.3.1
PyDrive2==1.6.3
pyerfa==2.0.1.1
pygame==2.5.2
Pygments==2.16.1
PyGObject==3.42.1
PyJWT==2.3.0
pymc==5.7.2
pymystem3==0.2.0
PyOpenGL==3.1.7
pyOpenSSL==24.0.0
pyparsing==3.1.1
pyperclip==1.8.2
pyproj==3.6.1
pyproject_hooks==1.0.0
pyshp==2.3.1
PySocks==1.7.1
pytensor==2.14.2
pytest==7.4.4
python-apt==0.0.0
python-box==7.1.1
python-dateutil==2.8.2
python-louvain==0.16
python-slugify==8.0.3
python-utils==3.8.2
pytz==2023.4
pyviz_comms==3.0.1
PyWavelets==1.5.0
PyYAML==6.0.1
pyzmq==23.2.1
qdldl==0.1.7.post0
qudida==0.0.4
ratelim==0.1.6
referencing==0.33.0
regex==2023.12.25
requests==2.31.0
requests-oauthlib==1.3.1
requirements-parser==0.5.0
rich==13.7.0
rpds-py==0.17.1
rpy2==3.4.2
rsa==4.9
safetensors==0.4.2
scikit-image==0.19.3
scikit-learn==1.2.2
scipy==1.11.4
scooby==0.9.2
scs==3.2.4.post1
seaborn==0.13.1
SecretStorage==3.3.1
Send2Trash==1.8.2
sentencepiece==0.1.99
shapely==2.0.2
six==1.16.0
sklearn-pandas==2.2.0
smart-open==6.4.0
sniffio==1.3.0
snowballstemmer==2.2.0
sortedcontainers==2.4.0
soundfile==0.12.1
soupsieve==2.5
soxr==0.3.7
spacy==3.7.2
spacy-legacy==3.0.12
spacy-loggers==1.0.5
Sphinx==5.0.2
sphinxcontrib-applehelp==1.0.8
sphinxcontrib-devhelp==1.0.6
sphinxcontrib-htmlhelp==2.0.5
sphinxcontrib-jsmath==1.0.1
sphinxcontrib-qthelp==1.0.7
sphinxcontrib-serializinghtml==1.1.10
SQLAlchemy==2.0.25
sqlglot==19.9.0
sqlparse==0.4.4
srsly==2.4.8
stanio==0.3.0
statsmodels==0.14.1
sympy==1.12
tables==3.8.0
tabulate==0.9.0
tbb==2021.11.0
tblib==3.0.0
tenacity==8.2.3
tensorboard==2.15.1
tensorboard-data-server==0.7.2
tensorflow==2.15.0
tensorflow-datasets==4.9.4
tensorflow-estimator==2.15.0
tensorflow-gcs-config==2.15.0
tensorflow-hub==0.16.1
tensorflow-io-gcs-filesystem==0.35.0
tensorflow-metadata==1.14.0
tensorflow-probability==0.23.0
tensorstore==0.1.45
termcolor==2.4.0
terminado==0.18.0
text-unidecode==1.3
textblob==0.17.1
tf-keras==2.15.0
tf-slim==1.1.0
thinc==8.2.2
threadpoolctl==3.2.0
tifffile==2024.1.30
tinycss2==1.2.1
tokenizers==0.15.1
toml==0.10.2
tomli==2.0.1
toolz==0.12.1
torch @ https://download.pytorch.org/whl/cu121/torch-2.1.0%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=0d4e8c52a1fcf5ed6cfc256d9a370fcf4360958fc79d0b08a51d55e70914df46
torch_geometric==2.4.0
torchaudio @ https://download.pytorch.org/whl/cu121/torchaudio-2.1.0%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=676bda4042734eda99bc59b2d7f761f345d3cde0cad492ad34e3aefde688c6d8
torchdata==0.7.0
torchsummary==1.5.1
torchtext==0.16.0
torchvision @ https://download.pytorch.org/whl/cu121/torchvision-0.16.0%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=e76e78d0ad43636c9884b3084ffaea8a8b61f21129fbfa456a5fe734f0affea9
tornado==6.3.2
tqdm==4.66.1
traitlets==5.7.1
traittypes==0.2.1
transformers==4.35.2
triton==2.1.0
tweepy==4.14.0
typer==0.9.0
types-pytz==2023.4.0.20240130
types-setuptools==69.0.0.20240125
typing_extensions==4.9.0
tzlocal==5.2
uc-micro-py==1.0.2
uritemplate==4.1.1
urllib3==2.0.7
vega-datasets==0.9.0
wadllib==1.3.6
wasabi==1.1.2
wcwidth==0.2.13
weasel==0.3.4
webcolors==1.13
webencodings==0.5.1
websocket-client==1.7.0
Werkzeug==3.0.1
widgetsnbextension==3.6.6
wordcloud==1.9.3
wrapt==1.14.1
xarray==2023.7.0
xarray-einstats==0.7.0
xgboost==2.0.3
xlrd==2.0.1
xxhash==3.4.1
xyzservices==2023.10.1
yarl==1.9.4
yellowbrick==1.5
yfinance==0.2.36
zict==3.0.0
zipp==3.17.0

#DREAM4 - MAIN

In [ ]:
from dig.xgraph.method import SubgraphX
from dig.xgraph.method import GNN_LRP
from torch_geometric.loader import DataLoader
from dig.xgraph.evaluation import XCollector
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx

In [6]:
''' CODE TO RUN AFTER PREPROCESSING '''
def main_run(folder_name,file_gold,hidden2_channels,out_channels,data,org,ds,dec,af_val,num_layers,epoch,aggr,var):
  auprs =0
  aucs =0

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
  for i in range(10) :
    # print("i is : ", i)
    split = T.RandomLinkSplit(
        num_val=0.2,
        num_test=0.3,
        is_undirected=False,
        add_negative_train_samples=True,
        key="edge_label"
    )

    train_data, val_data, test_data = split(data)

# 데이터 디바이스 이동
    train_data.x = train_data.x.to(device)
    train_data.edge_index = train_data.edge_index.to(device)
    train_data.edge_label = train_data.edge_label.to(device)
    train_data.edge_label_index = train_data.edge_label_index.to(device)

    val_data.x = val_data.x.to(device)
    val_data.edge_index = val_data.edge_index.to(device)
    val_data.edge_label = val_data.edge_label.to(device)
    val_data.edge_label_index = val_data.edge_label_index.to(device)

    test_data.x = test_data.x.to(device)
    test_data.edge_index = test_data.edge_index.to(device)
    test_data.edge_label = test_data.edge_label.to(device)
    test_data.edge_label_index = test_data.edge_label_index.to(device)

    index=(train_data.edge_label == 1).nonzero(as_tuple=True)[0]

    ################

    test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

    #####################

    ''' OLD CODE'''
    in_channels=data.num_features

    model =  Net(in_channels, hidden1_channels, hidden2_channels, out_channels,dec,af_val,num_layers,epoch,aggr,var).to(device)#Net(data.num_features, data.num_features, 128, 64).to(device) #self, in_channels, hidden1_channels, hidden2_channels,out_channels
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)#RMSprop(params=model.parameters())#
    criterion = torch.nn.BCEWithLogitsLoss()
    model = train_link_predictor(model, train_data, val_data, optimizer, criterion,epoch,af_val,dec).to(device)

    test_auc, precision, recall,fpr, tpr, mcc, jac_score, cohkap_score, f1, top_k = eval_link_predictor(model, test_data,af_val,dec)
    aucs = aucs+test_auc
    aupr = auc(recall, precision)
    auprs=auprs+aupr

    #####
    #  # 모델 출력 계산 (logits)
    # logits = model(test_data.x, test_data.edge_index, test_data.edge_label_index)
    # logits = logits.to(device)

    #   # 로짓 출력 확인
    # print("Logits Shape:", logits.shape)  # 예: (num_test_edges, 1)

    ######



  mean_auc = float(aucs/10)
  mean_aupr = float(auprs/10)


  list1 = [org, ds, dec, af_val,num_layers, epoch, aggr, var,mean_auc, mean_aupr, mcc, jac_score,cohkap_score, f1, top_k]
  df = pd.DataFrame(list1).T
  df.columns = ["org", "ds", "dec", "af_val","num_layers", "epoch", "aggr", "var","auc", "aupr", "mcc", "jac_score","cohkap_score", "f1", "top_k"]

# #########################
# # SubGraphX

#   explainer = SubgraphX(model, num_classes=2, device=device,
#                       explain_graph=False, reward_method='nc_mc_l_shapley')
# ########################
###########################
# GNN-LRP

  explainer = GNN_LRP(model, explain_graph=True)


  ###################

    # --- Set the Sparsity to 0. ---
  sparsity = 0.5
  num_classes = 2

  # --- Create data collector and explanation processor ---

  x_collector = XCollector(sparsity)
  # x_processor = ExplanationProcessor(model=model, device=device)
  #####################


  for index, data in enumerate(test_loader):
    print(f'explain graph line {test_loader.dataset.indices[index] + 2}')
    data.to(device)

    if torch.isnan(data.edge_label[0].squeeze()):
        continue

    walks, masks, related_preds = explainer(data.x, data.edge_index, sparsity=sparsity, num_classes=num_classes)

    x_collector.collect_data(masks, related_preds, data.edge_label[0].squeeze().long().item())

    # if you only have the edge masks without related_pred, please feed sparsity controlled mask to
    # obtain the result: x_processor(data, masks, x_collector)

    if index >= 1:
        break

##############################

  return df


In [7]:
gold_std = "./data/DREAM4/gold_std/"

InsilicoSize100_org  = {"Ecoli1":  ["InSilicoSize100","insilico_size100_1_knockdowns.tsv","insilico_size100_1_knockouts.tsv","insilico_size100_1_timeseries.tsv","DREAM4_GoldStandard_InSilico_Size100_1.tsv"],
                       "Ecoli2":  ["InSilicoSize100","insilico_size100_2_knockdowns.tsv","insilico_size100_2_knockouts.tsv","insilico_size100_2_timeseries.tsv","DREAM4_GoldStandard_InSilico_Size100_2.tsv"],
                       "Yeast1":  ["InSilicoSize100","insilico_size100_3_knockdowns.tsv","insilico_size100_3_knockouts.tsv","insilico_size100_3_timeseries.tsv","DREAM4_GoldStandard_InSilico_Size100_3.tsv"],
                       "Yeast2":  ["InSilicoSize100","insilico_size100_4_knockdowns.tsv","insilico_size100_4_knockouts.tsv","insilico_size100_4_timeseries.tsv","DREAM4_GoldStandard_InSilico_Size100_4.tsv"],
                       "Yeast3":  ["InSilicoSize100","insilico_size100_5_knockdowns.tsv","insilico_size100_5_knockouts.tsv","insilico_size100_5_timeseries.tsv","DREAM4_GoldStandard_InSilico_Size100_5.tsv"]
                            }


parameters = {
    "decoder": ["dot_sum"],# substitute "cos" to use cos decoder
    "af": ["F.silu"],  # substitute "F.sigmoid","F.tanh" for activation functions
    "num_layers" : [3],#2,4,5 #substitute layer count
    "variant" : ["HypergraphConv"],#,"SSGConv","ChebConv","ClusterGCNConv" #substitute convolution layers
    "aggrs" :["sum"],# substitute "add"
    "epochs" :[200]# substitute epochs 100,150,200,250
               }

hidden1_channels=128
hidden2_channels=64
out_channels= 32
num_layers = 4
final_out = pd.DataFrame()

ds_type =["basic_TS_aug_data_"]#,] # Try different graph types : "basic_TS_data_","basic_TS_aug_data_","basic_data_"
for org, files in InsilicoSize100_org.items():
  for ds in ds_type:
    for dec in parameters.get('decoder'):
      for lay_num in parameters.get('num_layers'):
        for epoch in parameters.get('epochs'):
          for af_val in parameters.get('af'):
            for aggr in parameters.get('aggrs'):
              for var in parameters.get('variant'):
                  print("current location : "+str(org)+'_'+str(ds)+'_'+str(dec)+'_'+str(lay_num)+'_'+str(epoch)+'_'+str(af_val)+'_'+str(aggr)+'_'+str(var))
                  data = torch.load("./data/DREAM4/DREAM4_InSilico_Size100/"+ds+org+"_"+files[0]+".pt")#.cuda()
                  temp = main_run(files[0],files[4],hidden2_channels,out_channels,data,org,ds,dec,af_val,lay_num,epoch,aggr,var)
                  final_out = pd.concat([final_out, temp], ignore_index=True)

print(final_out)

current location : Ecoli1_basic_TS_aug_data__dot_sum_3_200_F.silu_sum_HypergraphConv


KeyboardInterrupt: 

# SubGraph

In [1]:
from dig.xgraph.method import SubgraphX


In [ ]:
explainer = SubgraphX(model, num_classes=4, device=device,
                      explain_graph=False, reward_method='nc_mc_l_shapley')